# Свертка

In [1]:
import numpy as np
import imageio
import cv2

from matplotlib import pyplot as plt
%matplotlib inline

# Сверточная нейронная сеть

***Свёрточная нейросеть (Convolutional Neural Network, CNN)*** - это многослойная нейросеть, имеющая в своей архитектуре помимо полносвязных слоёв (а иногда их может и не быть) ещё и **свёрточные слои (Conv Layers)** и **pooling-слои (Pool Layers)**.  

Вот так выглядит неглубокая свёрточная нейросеть, имеющая такую архитектуру:  
`Input -> Conv 5x5 -> Pool 2x2 -> Conv 5x5 -> Pool 2x2 -> FC -> Output`

<img src="https://camo.githubusercontent.com/269e3903f62eb2c4d13ac4c9ab979510010f8968/68747470733a2f2f7261772e6769746875622e636f6d2f746176677265656e2f6c616e647573655f636c617373696669636174696f6e2f6d61737465722f66696c652f636e6e2e706e673f7261773d74727565" width=800>

Свёрточные нейросети почти всегда строятся по следующему правилу:  

`INPUT -> [[CONV -> RELU]*N -> POOL]*M -> [FC -> RELU]*K -> FC`  


1) ***Входной слой*** (batch картинок `HxWxC`)  

2) $M$ блоков из $N$ свёрток и pooling-ов, причём именно в том порядке, как в формуле выше. Все эти $M$ блоков вместе называют ***feature extractor*** свёрточной нейросети, потому что эта часть сети отвечает непосредственно за формирование новых, более сложных признаков, поверх тех, которые подаются.   

3) $K$ штук Fully-Connected-слоёв (с активациями). Эту часть из $K$ FC-слоёв называют ***classificator***, поскольку эти слои отвечают непосредственно за предсказание нужного класса.

Ещё раз вспомним про основные компоненты нейросети:

- сама **архитектура** нейросети (сюда входят типы функций активации у каждого нейрона);
- начальная **инициализация** весов каждого слоя;
- метод **оптимизации** нейросети (сюда ещё входит метод изменения `learning_rate`);
- размер **батчей** (`batch_size`);
- количетсво итераций обучения (`num_epochs`);
- **функция потерь** (`loss`);  
- тип **регуляризации** нейросети (для каждого слоя можно свой);  

Так как мы сейчас рассматриваем **CNN**, то, помимо этих компонент, в свёрточной нейросети можно настроить следующие параметры:  

- в каждом ConvLayer:
  - **размер фильтров (окна свёртки)** (`kernel_size`)
  - **количество фильтров** (`out_channels`)  
  - **шага окна свёртки (stride)** (`stride`)  
  - **тип padding'а** (`padding`)  


- в каждом PoolLayer:
  - **размер окна pooling'a** (`kernel_size`)  
  - **шаг окна pooling'а** (`stride`)  
  - **тип pooling'а** (`pool_type`)  
  - **тип padding'а** (`padding`)

Посмотрим, как работает CNN на датасетах MNIST и CIFAR10.

<img src="http://present5.com/presentation/20143288_415358496/image-8.jpg" width=500>

**MNIST:** это набор из 70k картинок рукописных цифр от 0 до 9, написанных людьми, 60k из которых являются тренировочной выборкой (`train` dataset)), и ещё 10k выделены для тестирования модели (`test` dataset).

<img src="https://raw.githubusercontent.com/soumith/ex/gh-pages/assets/cifar10.png" width=500>

**CIFAR10:** это набор из 60k картинок 32х32х3, 50k которых составляют обучающую выборку, и оставшиеся 10k - тестовую. Классов в этом датасете 10: `'plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'`.

In [2]:
import os

import numpy as np
import random
from tqdm import *

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
import torchvision.transforms as transforms

try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

import matplotlib.pyplot as plt
%matplotlib inline

[INFO] Couldn't find torchinfo... installing it.


In [3]:
# Зафиксируем seed для воспроизводимости

def seed_everything(seed):
    random.seed(seed) # фиксируем генератор случайных чисел
    os.environ['PYTHONHASHSEED'] = str(seed) # фиксируем заполнения хешей
    np.random.seed(seed) # фиксируем генератор случайных чисел numpy
    torch.manual_seed(seed) # фиксируем генератор случайных чисел pytorch
    torch.cuda.manual_seed(seed) # фиксируем генератор случайных чисел для GPU
    torch.backends.cudnn.deterministic = True # выбираем только детерминированные алгоритмы (для сверток)
    torch.backends.cudnn.benchmark = False # фиксируем алгоритм вычисления сверток

In [4]:
class CFG:

# Задаем параметры нашего эксперимента
  num_epochs = 10  # число эпох
  train_batch_size = 32  # размер батча для тренировки модели
  test_batch_size = 512  # размер батча для тестирования модели
  num_workers = 4  # число процессов для одновременной обработки данных (некритичный параметр при небольших датасетах)
  lr = 3e-4  # learning rate
  seed = 42  # random seed
  classes = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')  # классы из датасета CIFAR10


In [5]:
# Переведем наш класс с параметрами в словарь

def class2dict(f):
  return dict((name, getattr(f, name)) for name in dir(f) if not name.startswith('__'))

Построим нашу первую сверточную сеть LeNet5 (привет, Ян Лекун!)

In [6]:
class LeNet5(torch.nn.Module):
    def __init__(self,
                 activation='tanh',
                 pooling='avg',
                 conv_size=5
                ):
        super(LeNet5, self).__init__()

        self.conv_size = conv_size  # размер окна свёртки

        # установим на выбор функции активации
        if activation == 'tanh':
            activation_function = torch.nn.Tanh()
        elif activation == 'relu':
            activation_function = torch.nn.ReLU()
        else:
            raise NotImplementedError

        # установим на выбор тип пулинга
        if pooling == 'avg':
            pooling_layer = torch.nn.AvgPool2d(kernel_size=2, stride=2)
        elif pooling == 'max':
            pooling_layer = torch.nn.MaxPool2d(kernel_size=2, stride=2)
        else:
            raise NotImplementedError

        # установим на выбор размер ядра 1 слоя
        if conv_size == 5:
            self.conv1 = torch.nn.Conv2d(in_channels=1,
                                         out_channels=6,
                                         kernel_size=5,
                                         padding=2)
        elif conv_size == 3:
            self.conv1_1 = torch.nn.Conv2d(in_channels=1,
                                         out_channels=6,
                                         kernel_size=3,
                                         padding=1)
            self.conv1_2 = torch.nn.Conv2d(in_channels=6,
                                         out_channels=6,
                                         kernel_size=3,
                                         padding=1)
        else:
            raise NotImplementedError

        self.act1 = activation_function
        self.pool1 = pooling_layer

        # установим на выбор размер ядра 2 слоя
        if conv_size == 5:
            self.conv2 = self.conv2 = torch.nn.Conv2d(in_channels=6,
                                         out_channels=16,
                                         kernel_size=5,
                                         padding=0)
        elif conv_size == 3:
            self.conv2_1 = torch.nn.Conv2d(in_channels=6,
                                         out_channels=16,
                                         kernel_size=3,
                                         padding=0)
            self.conv2_2 = torch.nn.Conv2d(in_channels=16,
                                         out_channels=16,
                                         kernel_size=3,
                                         padding=0)
        else:
            raise NotImplementedError

        self.act2 = activation_function
        self.pool2 = pooling_layer

        # не забываем про полносвязанные слои
        self.fc1 = torch.nn.Linear(5 * 5 * 16, 120)
        self.act3 = activation_function

        self.fc2 = torch.nn.Linear(120, 84)
        self.act4 = activation_function

        self.fc3 = torch.nn.Linear(84, 10)

    # прямой проход
    def forward(self, x):
        # 1ый слой сети
        if self.conv_size == 5:
            x = self.conv1(x)
        elif self.conv_size == 3:
            x = self.conv1_2(self.conv1_1(x))

        x = self.act1(x)
        x = self.pool1(x)

        # 2ой слой сети
        if self.conv_size == 5:
            x = self.conv2(x)
        elif self.conv_size == 3:
            x = self.conv2_2(self.conv2_1(x))

        x = self.act2(x)
        x = self.pool2(x)

        # полносвязные слои сети
        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        x = self.act3(x)
        x = self.fc2(x)
        x = self.act4(x)
        x = self.fc3(x)

        return x

In [7]:
# функция обучения
def train(model, device, train_loader, optimizer, criterion, epoch):
    model.train() # переводим модель в режим обучения
    train_loss = 0  # инициализируем значение ошибки
    correct = 0  # иницилиазируем долю верно предсказанных объектов

    n_ex = len(train_loader)  # количество батчей в тренировочном датасете

    for batch_idx, (data, target) in tqdm(enumerate(train_loader), total=n_ex):
        data, target = data.to(device), target.to(device)  # отправляем данные на девайс (CPU или GPU), где находится модель

        optimizer.zero_grad() # обнуляем градиенты

        output = model(data)  # вызываем предсказание модели

        pred = output.argmax(dim=1, keepdim=True)  # получаем классы, который предсказала для каждого объекта

        correct += pred.eq(target.view_as(pred)).sum().item()  # считаем долю верно предсказанных объектов

        train_loss = criterion(output, target)  # считаем значение функции ошибки
        train_loss.backward() # обратное распространение функции ошибки (расчёт градиентов)
        optimizer.step() # делаем шаг оптимизатором (обновляем веса модели)

    tqdm.write('\nTrain set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        train_loss, 100. * correct / len(train_loader.dataset)))   # выводим результаты после одной эпохи

In [8]:
# функция инференса
def test(model, device, test_loader, criterion):
    model.eval() # переводим модель в режим тестирования
    train_loss = 0  # инициализируем значение ошибки
    correct = 0  # иницилиазируем долю верно предсказанных объектов

    with torch.no_grad(): # указываем, что не нужно считать градиенты, так как это режим тестирования
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)  # отправляем данные на девайс (CPU или GPU)
            output = model(data)  # вызываем предсказание модели
            test_loss = criterion(output, target)  # считаем значение функции ошибки
            pred = output.argmax(dim=1, keepdim=True)  # получаем классы, который предсказала для каждого объекта
            correct += pred.eq(target.view_as(pred)).sum().item()  # считаем долю верно предсказанных объектов

    tqdm.write('Test set: Average loss: {:.4f}, Accuracy: {:.2f}%'.format(
        test_loss, 100. * correct / len(test_loader.dataset)))   # выводим результаты после одной эпохи

In [13]:
def main_MNIST(model):

    use_cuda = torch.cuda.is_available() # проверяем, доступна ли нам GPU

    seed_everything(CFG.seed) # фиксируем random seed

    device = torch.device("cuda" if use_cuda else "cpu") # определяем, какой девайс нам доступен (GPU или CPU)

    kwargs = {'num_workers': CFG.num_workers, 'pin_memory': True} if use_cuda else {}

    # загружаем датасет MNIST
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,)) # значения среднего и стандартного отклонения
                       ])),
        batch_size=CFG.train_batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=CFG.test_batch_size, shuffle=False, **kwargs)

    model = model.to(device) # отправляем модель на девайс (GPU, если доступна, иначе CPU), где находится модель


    optimizer = optim.Adam(model.parameters(),
                          lr=CFG.lr) # инициализируем оптимизатор и отправляем ему веса модели и значение learning rate

    criterion = nn.CrossEntropyLoss() # инициализируем функцию ошибки

    for epoch in range(1, CFG.num_epochs + 1): # начинаем цикл обучения и тестирования
        print('\nEpoch:', epoch)
        train(model, device, train_loader, optimizer, criterion, epoch)  # обучение одной эпохи
        test(model, device, test_loader, criterion)  # тестирование после обучения одной эпохи
    print('Training is end!')

In [10]:
# будем сравнивать модели
model_1 = LeNet5(activation='tanh', conv_size=5)
model_2 = LeNet5(activation='relu', conv_size=5)
model_3 = LeNet5(activation='relu', conv_size=3)
model_4 = LeNet5(activation='relu', conv_size=3, pooling='max')

In [11]:
# Вывод информации о модели с помощью torchinfo
summary(model=model_4,
        input_size=(32, 1, 28, 28), # входной батч
        col_names=["input_size", "output_size", "num_params"], # что хотим посмотреть
        col_width=20
)

Layer (type:depth-idx)                   Input Shape          Output Shape         Param #
LeNet5                                   [32, 1, 28, 28]      [32, 10]             --
├─Conv2d: 1-1                            [32, 1, 28, 28]      [32, 6, 28, 28]      60
├─Conv2d: 1-2                            [32, 6, 28, 28]      [32, 6, 28, 28]      330
├─ReLU: 1-3                              [32, 6, 28, 28]      [32, 6, 28, 28]      --
├─MaxPool2d: 1-4                         [32, 6, 28, 28]      [32, 6, 14, 14]      --
├─Conv2d: 1-5                            [32, 6, 14, 14]      [32, 16, 12, 12]     880
├─Conv2d: 1-6                            [32, 16, 12, 12]     [32, 16, 10, 10]     2,320
├─ReLU: 1-7                              [32, 16, 10, 10]     [32, 16, 10, 10]     --
├─MaxPool2d: 1-8                         [32, 16, 10, 10]     [32, 16, 5, 5]       --
├─Linear: 1-9                            [32, 400]            [32, 120]            48,120
├─ReLU: 1-10                            

In [14]:
# запускаем цикл обучения и тестирования
main_MNIST(model_4)

100%|██████████| 9912422/9912422 [00:00<00:00, 39635362.09it/s]


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 111440380.70it/s]


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 43487146.67it/s]

...

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
100%|██████████| 1875/1875 [00:20<00:00, 91.04it/s] 


Train set: Average loss: 0.1022, Accuracy: 91.94%


Test set: Average loss: 0.1918, Accuracy: 97.02%

Epoch: 2


 51%|█████     | 957/1875 [00:10<00:08, 104.16it/s]Exception in thread Thread-13 (_pin_memory_loop):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
 51%|█████     | 960/1875 [00:10<00:10, 90.60it/s] 
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 54, in _pin_memory_loop
    do_one_step()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/_utils/pin_memory.py", line 31, in do_one_step
    r = in_queue.get(timeout=MP_STATUS_CHECK_INTERVAL)
  File "/usr/lib/python3.10/multiprocessing/queues.py", line 122, in get
    return _ForkingPickler.loads(res)
  File "/usr/local/lib/python3.10/dist-packages/torch/multiprocessing/reductions.py", line 355, in rebuild_storage_fd
    fd = df.detach()
  File "/usr/lib/python3.10/multiprocessing/resource_sharer.py", 

KeyboardInterrupt: ignored

Давайте теперь устроим небольшой поединок между полносвязанной сетью и сверточной на датасете CIFAR10.

In [15]:
def main_CIFAR(model):

    use_cuda = torch.cuda.is_available()

    seed_everything(CFG.seed)

    device = torch.device("cuda" if use_cuda else "cpu")

    kwargs = {'num_workers': CFG.num_workers, 'pin_memory': True} if use_cuda else {}

    # загружаем датасет CIFAR10
    train_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)) # нормализуем значения
                       ])),
        batch_size=CFG.train_batch_size, shuffle=True, **kwargs)

    test_loader = torch.utils.data.DataLoader(
        datasets.CIFAR10('../data', train=False, transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261))
                       ])),
        batch_size=CFG.test_batch_size, shuffle=False, **kwargs)

    model = model.to(device)


    optimizer = optim.Adam(model.parameters(),
                          lr=CFG.lr)

    criterion = nn.CrossEntropyLoss()

    for epoch in range(1, CFG.num_epochs + 1):
        print('\nEpoch:', epoch)
        train(model, device, train_loader, optimizer, criterion, epoch)
        test(model, device, test_loader, criterion)
    print('Training is end!')

In [16]:
# определяем полносвязанную сеть
class FC_Net(nn.Module):
    def __init__(self, constant_weight=None, normal=False,
                 xavier_uniform=False, he_normal=False):
        super(FC_Net, self).__init__()

        self.fc1 = nn.Linear(3072, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 10)

    def forward(self, x):
        x = x.view(-1, 3072)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)

        return x

In [17]:
model_1 = FC_Net()

In [18]:
main_CIFAR(model_1)

100%|██████████| 170498071/170498071 [00:05<00:00, 30130689.97it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(



Epoch: 1


100%|██████████| 1563/1563 [00:16<00:00, 94.29it/s]

...


Train set: Average loss: 0.8042, Accuracy: 75.50%


Test set: Average loss: 1.8095, Accuracy: 53.76%

Epoch: 10


100%|██████████| 1563/1563 [00:19<00:00, 78.27it/s]


Train set: Average loss: 0.5739, Accuracy: 78.25%


Test set: Average loss: 1.7625, Accuracy: 55.04%
Training is end!


Видим, что инициализация весов крайне важна для успешного обучения сети. Наглядно увидеть, как от инициализации весов зависит протекание градиентов можно тут https://www.deeplearning.ai/ai-notes/initialization/index.html  

In [ ]:
# создаем сверточную сеть для CIFAR10
class CIFAR_Net(torch.nn.Module):
    def __init__(self):
        super(CIFAR_Net, self).__init__()

        self.conv1 = torch.nn.Conv2d(3, 16, 3, padding=1)
        self.act1  = torch.nn.ReLU()
        self.pool1 = torch.nn.MaxPool2d(2, 2)

        self.conv2 = torch.nn.Conv2d(16, 32, 3, padding=1)
        self.act2  = torch.nn.ReLU()
        self.pool2 = torch.nn.MaxPool2d(2, 2)

        self.conv3 = torch.nn.Conv2d(32, 64, 3, padding=1)
        self.act3  = torch.nn.ReLU()

        self.fc1   = torch.nn.Linear(8 * 8 * 64, 256)
        self.act4  = torch.nn.Tanh()

        self.fc2   = torch.nn.Linear(256, 64)
        self.act5  = torch.nn.Tanh()

        self.fc3   = torch.nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.act1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool2(x)

        x = self.conv3(x)
        x = self.act3(x)

        x = x.view(x.size(0), x.size(1) * x.size(2) * x.size(3))
        x = self.fc1(x)
        x = self.act4(x)
        x = self.fc2(x)
        x = self.act5(x)
        x = self.fc3(x)

        return x

In [ ]:
model_CNN = CIFAR_Net()

In [ ]:
summary(model=model_CNN,
        input_size=(32, 3, 32, 32), # входной батч
        col_names=["input_size", "output_size", "num_params"], # что хотим посмотреть
        col_width=20
)

Layer (type:depth-idx)                   Input Shape          Output Shape         Param #
CIFAR_Net                                [32, 3, 32, 32]      [32, 10]             --
├─Conv2d: 1-1                            [32, 3, 32, 32]      [32, 16, 32, 32]     448
├─ReLU: 1-2                              [32, 16, 32, 32]     [32, 16, 32, 32]     --
├─MaxPool2d: 1-3                         [32, 16, 32, 32]     [32, 16, 16, 16]     --
├─Conv2d: 1-4                            [32, 16, 16, 16]     [32, 32, 16, 16]     4,640
├─ReLU: 1-5                              [32, 32, 16, 16]     [32, 32, 16, 16]     --
├─MaxPool2d: 1-6                         [32, 32, 16, 16]     [32, 32, 8, 8]       --
├─Conv2d: 1-7                            [32, 32, 8, 8]       [32, 64, 8, 8]       18,496
├─ReLU: 1-8                              [32, 64, 8, 8]       [32, 64, 8, 8]       --
├─Linear: 1-9                            [32, 4096]           [32, 256]            1,048,832
├─Tanh: 1-10                      

In [ ]:
main_CIFAR(model_CNN)

100%|██████████| 170498071/170498071 [00:03<00:00, 43686862.75it/s]


Extracting ../data/cifar-10-python.tar.gz to ../data

Epoch: 1


100%|██████████| 1563/1563 [00:18<00:00, 84.58it/s]



Train set: Average loss: 1.4697, Accuracy: 48.42%
Test set: Average loss: 1.2576, Accuracy: 58.12%

Epoch: 2


100%|██████████| 1563/1563 [00:20<00:00, 77.84it/s]

...


Train set: Average loss: 0.3853, Accuracy: 89.41%


Test set: Average loss: 0.9634, Accuracy: 72.61%

Epoch: 10


100%|██████████| 1563/1563 [00:18<00:00, 83.56it/s]


Train set: Average loss: 0.2271, Accuracy: 92.35%


Test set: Average loss: 1.0333, Accuracy: 72.51%
Training is end!


Итак, приходится признать очевидный факт: сверточные сети гораздо лучше решают задачу классификации изображений, нежели полносвязанные.